In [ ]:
import math
import torch
import gpytorch
from matplotlib import pyplot as plt

import pandas as pd
import numpy as np


%matplotlib inline
%load_ext autoreload
%autoreload 2

###  Loading and pre-processing the data

In [ ]:
train_data = pd.read_pickle("/share/rcifdata/jbarr/UKAEAGroupProject/data/QLKNN_train_data.pkl")

In [ ]:
# choosing one input dimension and one output dimension as random
input_dim = np.random.permutation(list(train_data.iloc[:,:15].columns))
output_dim = np.random.permutation(list(train_data.iloc[:,15:].columns))

In [ ]:
print(f"Input dimension to use: {input_dim[0]} and {input_dim[1]}")
print(f"Input dimension to use: {output_dim[0]}")

In [ ]:
drop_data = train_data[[f'{input_dim[0]}',f'{input_dim[1]}',f'{output_dim[0]}']].dropna()

In [ ]:
x_train_data = drop_data.iloc[:,:2]
y_train_data = drop_data.iloc[:,2:]

assert x_train_data.shape[0] == y_train_data.shape[0]

In [ ]:
n = 100
idx = np.random.permutation(n)

In [ ]:
x_train_data = torch.tensor(x_train_data.values)[idx]
y_train_data = torch.tensor(y_train_data.values)[idx]

x_min, x_max = x_train_data.min(), x_train_data.max()

In [ ]:
# We will use the simplest form of GP model, exact inference
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel(ard_num_dims=2), num_dims=2)

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

In [ ]:
# initialize likelihood and model
likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = ExactGPModel(x_train_data, y_train_data, likelihood)

In [ ]:
# this is for running the notebook in our testing framework
import os
smoke_test = ('CI' in os.environ)
training_iter = 2 if smoke_test else 100


# Find optimal model hyperparameters
model.train()
likelihood.train()

# Use the adam optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)  # Includes GaussianLikelihood parameters

# "Loss" for GPs - the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

for i in range(training_iter):
    # Zero gradients from previous iteration
    optimizer.zero_grad()
    # Output from model
    output = model(x_train_data)
    # Calc loss and backprop gradients
    loss = -mll(output, y_train_data).mean()
    loss.backward()
    
    lenlist = model.covar_module.base_kernel.lengthscale.tolist()[0]
    
    noise = model.likelihood.noise.item()
    
    lengthscales = [f'{lenlist[0]:.3f}', f'{lenlist[1]:.3f}']
    
    print(f'Iter {(i+1)/training_iter} - Loss: {loss.item():.3f} lengthscales:{lengthscales} noise: {noise:.4f} ')
    
    optimizer.step()

In [ ]:
# Get into evaluation (predictive posterior) mode
model.eval()
likelihood.eval()

# Test points are regularly spaced along [0,1]
# Make predictions by feeding model through likelihood
with torch.no_grad(), gpytorch.settings.fast_pred_var():
    testx1 = np.linspace(x_min, x_max , 50)
    testx2 =  np.linspace(x_min, x_max , 50)
    
    test_x = torch.tensor(np.array(np.meshgrid(testx1, testx2)).T.reshape(-1,2), dtype = torch.double)
    
    print(x_train_data.shape)
    print(test_x.shape)
    
    observed_pred = likelihood(model(test_x))

In [ ]:
with torch.no_grad():
    # Initialize plot
    f, ax = plt.subplots(1, 1, figsize=(8, 6))

    # Get upper and lower confidence bounds
    lower, upper = observed_pred.confidence_region()
    # Plot training data as black stars
    ax.plot(x_train_data.numpy(), y_train_data.numpy(), 'k*')
    # Plot predictive means as blue line
    ax.plot(test_x, observed_pred.mean.numpy(), 'b')
    # Shade between the lower and upper confidence bounds
    ax.fill_between(test_x.numpy(), lower.numpy(), upper.numpy(), alpha=0.5)
#     ax.set_ylim([-3, 3])
    ax.legend(['Observed Data', 'Mean', 'Confidence'])